# Motion segmentation

In [1]:
# For Kmeans algorithm. See PGM otherwise for handcrafted class
from sklearn.cluster import KMeans
# For loading matlab matrix file
from scipy.io import loadmat
# Hungarian algorithm
from scipy.optimize import linear_sum_assignment

#from scipy.sparse.linalg import eigs, eigsh
from scipy.stats import itemfreq
import numpy as np

In [99]:
# %load_ext line_profiler
# %lprun -f function_to_profile code_to_execute

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
%precision %.5f
plt.rc('text', usetex=True)
plt.rc('font',**{'family':'serif','serif':['Palatino']})

In [3]:
#Algorithm and error
from error_evaluation import *
from Ksubspace import *
from spectral_clustering import *
from SSC import *

Extrait de 

@article{elhamifar2013sparse,

  title={Sparse subspace clustering: Algorithm, theory, and applications},
  
  author={Elhamifar, Ehsan and Vidal, Rene},
  
  journal={IEEE transactions on pattern analysis and machine intelligence},
  
  volume={35},
  
  number={11},
  
  pages={2765--2781},
  
  year={2013},
  
  publisher={IEEE}
  
}


![article](real_data_article.png)

## Globals

In [4]:
DATA_DIR = "data/Hopkins155/"
NEEDLE = "_truth.mat"

## Data

In [5]:
import glob
import os

In [90]:
motion_data = {}
count_files = 0
# iglob for iterator
## Recursive=True to use **
for file in glob.iglob(DATA_DIR + "**/*" + NEEDLE, recursive = True):
    mat_name = file.split("/")[-1].split(NEEDLE)[0]
    try:
        # Load matlab matrix
        motion_data[mat_name] = loadmat(file)
        # Cast in float64 x
        motion_data[mat_name]['x'] = motion_data[mat_name]\
        ['x'].astype(np.float64, subok = True, copy = False)
        # Cast number of points in int
        motion_data[mat_name]['points'] = motion_data[mat_name]\
        ['points'][0, 0].astype(int)
        # Cast number of frames in int
        motion_data[mat_name]['frames'] = motion_data[mat_name]\
        ['frames'][0, 0].astype(int)
        # Substract 1 to matlab labels
        motion_data[mat_name]['s'] -= 1
    except OSError:
        print ('cannonot open ', file)
    else:
        count_files += 1
        motion_data[mat_name]["data"] = \
        np.transpose(motion_data[mat_name]['x'][:2, : :],
                     axes = (0, 2, 1)).reshape(2 * motion_data[mat_name]['frames'],
                                               motion_data[mat_name]["points"])
print ("Number of files loaded : {}".format(count_files))

Number of files loaded : 156


# Test zone

## Cars1

|$\tau$ | $\mu_2$| $error (\%)$|
|-------|--------|-------------|
|1e6    |1e6     |1.30         |
|1e8    |1e6     |16.94        |
|1e3    |1e6     |0.00         |
|1e3    |1e4     |0.32         |
|    |     |         |
|    |     |         |
|    |     |         |
|    |     |         |
|    |     |         |
|    |     |         |
|    |     |         |
|    |     |         |


In [7]:
key = "cars1"

In [8]:
# Data
motion_data[key]["data"].shape

(40, 307)

In [9]:
# Labels
print ("Number of groups : {}".format(motion_data[key]["s"].max() + 1))
motion_data[key]["s"].shape

Number of groups : 2


(307, 1)

In [ ]:
for tau in np.arange()

In [28]:
T = SSC(motion_data["cars1"]["data"], 2, 10e2, 10e5)

0.791825203389 0.795243378578 70773.0393451
0.301075679544 0.320356227007 40321.8191709
0.169393182471 0.171502454428 14257.4253615
0.119182984684 0.119483152801 6412.92716083
0.0898851563685 0.0899278951663 3833.39049027
0.0708234748985 0.0708344032005 2587.6366156
0.05823515058 0.0582389492787 1822.12498294
0.0497988087144 0.0498007290195 1310.85918293
0.0439450608337 0.0439462754426 960.639978544
0.0396583211328 0.0396585892914 719.099254287
0.0363190849748 0.0363195448082 552.615655984
0.0335678701018 0.0335678122945 437.628420779
0.0312009261632 0.0312008858977 358.193070034
0.0291039969701 0.0291037475896 302.135331917
0.0272113260299 0.0272114652238 261.862287095
0.0254843658494 0.025484442543 232.029041049
0.0238991632332 0.023898999742 209.056347075
0.0224393011382 0.0224391850928 190.238347058
0.0210926909016 0.0210926456665 174.589200107
0.019849911515 0.0198497604115 160.934432034
0.0187024936094 0.0187027854725 149.583575588
0.0176438669292 0.0176437081267 138.739958406
0.

In [27]:
# 10e5 10e5 : 0.01303
evaluate_error(T[1], motion_data["cars1"]["s"].reshape(-1))

Erreur : 0.003257328990228013


0.00326

In [24]:
T[1]

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0,

## 2RT3RC

|$\tau$ | $\mu_2$| $error (\%)$|
|-------|--------|-------------|
|1e3    |1e6     |48.46         |
|1e5    |1e6     |23.41         |
|1e5    |1e3     |2.36         |
|1e2    |     |    Not converging     |
|1e6    |1e1     |40.47         |
|1e6    |1e3     |1.63         |
|1e6    |1e4     |1.63         |
|1e6    |1e5     |2.90       |
|1e6    |1e8    |35.57         |
|1e7    |1e3     |1.63         |
|    |     |         |

In [96]:
key = "2RT3RC"
T = SSC(motion_data[key]["data"], motion_data[key]["s"].max() + 1, 1e6, 1e3)
evaluate_error(T[1], motion_data[key]["s"].reshape(-1))

[[ 3.08478096  0.          0.         ...,  0.          0.          0.        ]
 [ 0.          4.51525604  0.         ...,  0.          0.          0.        ]
 [ 0.          0.          2.44128282 ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  7.40659508  0.          0.        ]
 [ 0.          0.          0.         ...,  0.          1.58260061  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.99505101]]
Erreur : 0.016333938294010888


0.01633

In [98]:
%lprun -f  compute_sparse_C T = SSC(motion_data[key]["data"], motion_data[key]["s"].max() + 1, 1e7, 1e3)

[[  3.14557565   0.           0.         ...,   0.           0.           0.        ]
 [  0.           8.01983806   0.         ...,   0.           0.           0.        ]
 [  0.           0.           2.78783241 ...,   0.           0.           0.        ]
 ..., 
 [  0.           0.           0.         ...,  16.18052198   0.           0.        ]
 [  0.           0.           0.         ...,   0.           1.31183943
    0.        ]
 [  0.           0.           0.         ...,   0.           0.
    3.39565212]]


In [100]:
key = "2RT3RC"
nbr_subspaces = motion_data[key]["s"].max() + 1
T = ksubspaces(motion_data[key]["data"], nbr_subspaces , 5 * np.ones((nbr_subspaces, 1)) ,3)
evaluate_error(T[1], motion_data[key]["s"].reshape(-1))

mu Error :  14.7760102307 U error 29.7572831966
mu Error :  2.50843145858 U error 5.46955814353
mu Error :  1.05445993814 U error 0.195436054088
mu Error :  0.660819037334 U error 2.00911535602
mu Error :  0.374601049172 U error 0.14359202168
mu Error :  0.180515827573 U error 0.163880877335
mu Error :  0.0752819960232 U error 2.81928112257
mu Error :  0.209993977063 U error 2.00100783599
mu Error :  0.0972488188208 U error 0.136523148484
mu Error :  0.0333517418146 U error 0.0770138446432
mu Error :  0.0511123638619 U error 0.0449493111585
mu Error :  0.0473373469231 U error 0.0885027766651
mu Error :  0.0485126513663 U error 0.0825816494277
mu Error :  0.00988463020216 U error 0.0247000456797
mu Error :  0.0203614254004 U error 0.0307777618991
mu Error :  0.021879912918 U error 0.0375758653168
mu Error :  0.0 U error 0.0
mu Error :  9.18170819583 U error 29.5883448503
mu Error :  4.03166188158 U error 5.28160483255
mu Error :  1.18753548028 U error 2.50369772127
mu Error :  1.7158479

ValueError: Lengths of clustering do no match